# PowerShell Encoded Command

**PowerShell executed with -enc/-encodedcommand or Base64 decode primitives, often indicating obfuscation/malware.**

This notebook is a **ready-to-use SOC playbook** with:

- Triage checklist
- Splunk SPL queries
- Microsoft Sentinel KQL queries
- Elastic KQL/EQL concepts
- Containment & response commands (PowerShell / Windows / MDE)

> Replace placeholders like `<USER>`, `<HOST>`, `<DEVICE_ID>`, `<TENANT_DOMAIN>` before running.


## Triage Checklist

**Collect:** user/account, host/device, source IP, time window, impacted resources, and any correlated alerts.

**Validate:** baseline/allowlists (VPN egress, jump hosts, admin tools) before escalating.

**Preserve:** keep a copy of key logs and process trees before remediation when possible.

Focus: **parent process**, **download source**, **user context**, **script block logging/AMSI**, **network beacons**.


## Investigation Queries

Find the command line and parent chain, then hunt for related processes and network connections.


In [ ]:
// KQL
DeviceProcessEvents
| where TimeGenerated > ago(7d)
| where DeviceName == '<HOST>'
| where FileName in~ ('powershell.exe','pwsh.exe')
| where ProcessCommandLine has_any ('-enc','-encodedcommand','FromBase64String')
| project TimeGenerated, DeviceName, AccountName, InitiatingProcessFileName, ProcessCommandLine, SHA256


In [ ]:
# Splunk SPL
index=endpoint (Image=*\\powershell.exe OR Image=*\\pwsh.exe)
(CommandLine="*-enc *" OR CommandLine="*-encodedcommand *" OR CommandLine="*FromBase64String*")
| table _time host user Image CommandLine ParentImage


In [ ]:
# Elastic (KQL/EQL)
process.name:(powershell.exe OR pwsh.exe) AND process.command_line:(*"-enc"* OR *"-encodedcommand"* OR *"FromBase64String"*)


## Containment & Response

If malicious: isolate host, kill process tree, block hash, and collect forensics.


In [ ]:
# PowerShell
## Kill suspicious PowerShell (local)
Get-Process powershell,pwsh -ErrorAction SilentlyContinue | Stop-Process -Force


In [ ]:
# PowerShell
## Microsoft Defender for Endpoint isolation (conceptual; requires permissions/tools)
# Invoke-MDATPDeviceIsolation -MachineId '<DEVICE_ID>'


In [ ]:
# Bash
# Collect triage artifacts (Windows)
# wevtutil epl Security C:\IR\Security.evtx
# wevtutil epl Microsoft-Windows-Sysmon/Operational C:\IR\Sysmon.evtx
